# Multimodal RAG app with LanceDB and LlamaIndex

## Video Processing

In [5]:
from moviepy import VideoFileClip
from pathlib import Path
import speech_recognition as sr
from pytube import YouTube
from pprint import pprint
from PIL import Image
import matplotlib.pyplot as plt